In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import pretty_midi as pyd
from arrangement_utils import *
import warnings
warnings.filterwarnings("ignore")

"""Download checkpoints and data at https://drive.google.com/drive/folders/17yB-Oae_4eGKJmqRS-LB8PwE2rqwZrUu?usp=sharing (582MB) and decompress at the directory"""
DATA_FILE_ROOT = './data_file_dir/'
DEVICE = 'cuda:0'
piano_arranger, orchestrator, piano_texture, band_prompt = load_premise(DATA_FILE_ROOT, DEVICE)

/home/zhaojw/anaconda3/envs/torch1.12/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhaojw/anaconda3/envs/torch1.12/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Loading AccoMontage piano texture search space. This may take 1 or 2 minutes ...


100%|██████████| 11/11 [00:02<00:00,  4.50it/s]


loading orchestration prompt search space ...
loading Slakh2100 Dataset ...


100%|██████████| 214/214 [00:10<00:00, 20.48it/s]


Initialize model ...
Finished.


### Initialize input and preference
We provide four sample lead sheets for a quick inference. You should be able to directly run the code blocks after downloading the pre-trained checkpoints.

If you wish to test our model on your own lead sheet file, please initialize a sub-folder with its `SONG_NAME` in the `./demo` folder and put the file in, and name the file "lead sheet.mid". 

Please also specify `SEGMENTATION` (phrase structure) and `NOTE_SHIFT` (the duration of the pick-up measure if any).

In [14]:
"""Set input lead sheet"""
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = 'Castles in the Air', 'A8A8B8B8', 1 , 100  #1 beat in the pick-up measure
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = 'Jingle Bells', 'A8B8A8', 0, 100
#SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = 'Sally Garden', 'A4A4B4A4', 0, 75
SONG_NAME, SEGMENTATION, NOTE_SHIFT, TEMPO = 'Auld Lang Syne', 'A8B8A8B8', 1, 80

"""Set texture pre-filtering for piano arrangement (default random)"""
RHTHM_DENSITY = np.random.randint(2, 5)
VOICE_NUMBER = np.random.randint(2, 5)
PREFILTER = (RHTHM_DENSITY, VOICE_NUMBER)

"""Set if use a 2-bar prompt for full-band arrangement (default True)""" 
USE_PROMPT = True

lead_sheet = read_lead_sheet('./demo', SONG_NAME, SEGMENTATION, NOTE_SHIFT)

### Piano Accompaniment Arrangement

In [15]:
midi_piano, acc_piano = piano_arrangement(*lead_sheet, *piano_texture, piano_arranger, PREFILTER, TEMPO)
midi_piano.write(f'./demo/{SONG_NAME}/arrangement_piano.mid')

Phrasal Unit selection begins:
	 4 phrases in the lead sheet;
	 set note density filter: (3, 3).


100%|██████████| 3/3 [00:10<00:00,  3.48s/it]


Re-harmonization begins ...
Piano accompaiment generated!


### Orchestration

In [17]:
func_prompt = prompt_sampling(acc_piano, *band_prompt, DEVICE)

Prior model initialized with 6 tracks:
	['Soprano/Alto Sax', 'Choir and Voice', 'Clean Electric Guitar', 'Acoustic Guitar', 'Acoustic Piano', 'Electric Bass']


In [18]:
if USE_PROMPT:
    midi_band = orchestration(acc_piano, None, *func_prompt, orchestrator, DEVICE, blur=.25, p=.05, t=6, tempo=TEMPO)
else:
    instruments, time_promt = func_prompt
    midi_band = orchestration(acc_piano, None, instruments, None, orchestrator, DEVICE, blur=.25, p=.05, t=8, tempo=TEMPO)
mel_track = pyd.Instrument(program=72, is_drum=False, name='melody')
mel_track.notes = midi_piano.instruments[0].notes
midi_band.instruments.append(mel_track)
midi_band.write(f'./demo/{SONG_NAME}/arrangement_band.mid')

Orchestration begins ...
Full-band accompaiment generated!
